In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pyodbc
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# Função para buscar as credenciais do Key Vault
def retorna_credenciais(secret_name):
    KEY_VAULT_URL = "https://kv-academy-01.vault.azure.net/"
    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KEY_VAULT_URL, credential=credential)
    retrieved_secret = client.get_secret(secret_name)
    return retrieved_secret.value

# Obter credenciais seguras
server = retorna_credenciais("db-academy-server")
database = retorna_credenciais("db-academy-database")
username = retorna_credenciais("db-academy-user")
password = retorna_credenciais("db-academy-password")
driver = '{ODBC Driver 18 for SQL Server}'

# Montar string de conexão usando pyodbc
connection_string = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)

# Criar a conexão com o banco de dados
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("Conexão com o banco de dados bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    raise e

# Função para popular a tabela de resultados
def populate_table(df):
    for row in result_table.get_children():
        result_table.delete(row)

    if not df.empty:
        for _, row in df.iterrows():
            result_table.insert("", "end", values=list(row))
    else:
        messagebox.showinfo("Resultado", "Nenhum resultado encontrado.")

# Funções CRUD
def create_transaction():
    try:
        query = f"""
        INSERT INTO dbo.projeto_daniel_danillo_matheus 
        (Tempo, Tipo, Valor, ContaOrigem, SaldoAntigoOrigem, SaldoNovoOrigem, ContaDestino, SaldoAntigoDestino, SaldoNovoDestino, Fraude)
        VALUES ({tempo.get()}, '{tipo.get()}', {valor.get()}, '{conta_origem.get()}', {saldo_antigo_origem.get()}, {saldo_novo_origem.get()},
                '{conta_destino.get()}', {saldo_antigo_destino.get()}, {saldo_novo_destino.get()}, {fraude.get()})
        """
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação criada com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def read_transactions():
    try:
        query = "SELECT * FROM dbo.projeto_daniel_danillo_matheus"
        user_condition = condition_read.get().strip()

        if user_condition:
            query += f" WHERE {user_condition}"

        df = pd.read_sql(query, conn)
        populate_table(df)
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def fetch_last_10_records():
    try:
        query = "SELECT TOP 10 * FROM dbo.projeto_daniel_danillo_matheus ORDER BY Tempo DESC"
        df = pd.read_sql(query, conn)
        populate_table(df)
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def update_transaction():
    try:
        update_str = f"{coluna.get()} = '{novo_valor.get()}'"
        query = f"UPDATE dbo.projeto_daniel_danillo_matheus SET {update_str} WHERE {condition_update.get()}"
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação atualizada com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def delete_transaction():
    try:
        query = f"DELETE FROM dbo.projeto_daniel_danillo_matheus WHERE {condition_delete.get()}"
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação excluída com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

# Configuração da interface gráfica
root = tk.Tk()
root.title("CRUD - Tkinter")
root.geometry("1200x600")

notebook = ttk.Notebook(root)
notebook.pack(fill=tk.BOTH, expand=True)

create_frame = ttk.Frame(notebook)
read_frame = ttk.Frame(notebook)
update_frame = ttk.Frame(notebook)
delete_frame = ttk.Frame(notebook)

notebook.add(create_frame, text="Create")
notebook.add(read_frame, text="Read")
notebook.add(update_frame, text="Update")
notebook.add(delete_frame, text="Delete")

# Aba Create
tk.Label(create_frame, text="Tempo").grid(row=0, column=0)
tempo = tk.Entry(create_frame)
tempo.grid(row=0, column=1)

tk.Label(create_frame, text="Tipo").grid(row=1, column=0)
tipo = tk.Entry(create_frame)
tipo.grid(row=1, column=1)

tk.Label(create_frame, text="Valor").grid(row=2, column=0)
valor = tk.Entry(create_frame)
valor.grid(row=2, column=1)

tk.Label(create_frame, text="Conta Origem").grid(row=3, column=0)
conta_origem = tk.Entry(create_frame)
conta_origem.grid(row=3, column=1)

tk.Label(create_frame, text="Saldo Antigo Origem").grid(row=4, column=0)
saldo_antigo_origem = tk.Entry(create_frame)
saldo_antigo_origem.grid(row=4, column=1)

tk.Label(create_frame, text="Saldo Novo Origem").grid(row=5, column=0)
saldo_novo_origem = tk.Entry(create_frame)
saldo_novo_origem.grid(row=5, column=1)

tk.Label(create_frame, text="Conta Destino").grid(row=6, column=0)
conta_destino = tk.Entry(create_frame)
conta_destino.grid(row=6, column=1)

tk.Label(create_frame, text="Saldo Antigo Destino").grid(row=7, column=0)
saldo_antigo_destino = tk.Entry(create_frame)
saldo_antigo_destino.grid(row=7, column=1)

tk.Label(create_frame, text="Saldo Novo Destino").grid(row=8, column=0)
saldo_novo_destino = tk.Entry(create_frame)
saldo_novo_destino.grid(row=8, column=1)

tk.Label(create_frame, text="Fraude").grid(row=9, column=0)
fraude = tk.Entry(create_frame)
fraude.grid(row=9, column=1)

tk.Button(create_frame, text="Criar Transação", command=create_transaction).grid(row=10, column=0, columnspan=2)

# Aba Read
tk.Label(read_frame, text="Condição (opcional)").pack(pady=5)
condition_read = tk.Entry(read_frame)
condition_read.pack(pady=5)

tk.Button(read_frame, text="Ler Transações", command=read_transactions).pack(pady=10)

# Configurar tabela com todas as colunas
result_table = ttk.Treeview(read_frame, show="headings", height=10)
result_table.pack(fill=tk.BOTH, expand=True, pady=10)

columns = [
    "Tempo", "Tipo", "Valor", "ContaOrigem", "SaldoAntigoOrigem", "SaldoNovoOrigem", 
    "ContaDestino", "SaldoAntigoDestino", "SaldoNovoDestino", "Fraude"
]
result_table["columns"] = columns

for col in columns:
    result_table.heading(col, text=col)
    result_table.column(col, width=120)

scrollbar = ttk.Scrollbar(read_frame, orient="vertical", command=result_table.yview)
result_table.configure(yscroll=scrollbar.set)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Aba Update
tk.Label(update_frame, text="Coluna").pack()
coluna = tk.Entry(update_frame)
coluna.pack()

tk.Label(update_frame, text="Novo Valor").pack()
novo_valor = tk.Entry(update_frame)
novo_valor.pack()

tk.Label(update_frame, text="Condição").pack()
condition_update = tk.Entry(update_frame)
condition_update.pack()

tk.Button(update_frame, text="Atualizar Transação", command=update_transaction).pack()

# Aba Delete
tk.Label(delete_frame, text="Condição").pack()
condition_delete = tk.Entry(delete_frame)
condition_delete.pack()

tk.Button(delete_frame, text="Excluir Transação", command=delete_transaction).pack()

# Carregar os últimos 10 registros ao iniciar
fetch_last_10_records()

# Iniciar o loop principal
root.mainloop()
